# 4. Problemi u Treniranju Modela

## Identifikacija i analiza neuspešnog treniranja

---

Nakon implementacije MultiOutputCarClassifier modela, pokušan je trening sa two-phase pristupom. Međutim, rezultati su pokazali nekoliko kritičnih problema koji su onemogućili uspešno učenje."

In [1]:
# Import biblioteka za analizu
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

### Rezultati neuspešnog treniranja

Trening sa standardnim MultiOutputCarClassifier modelom je završen sa sledećim rezultatima:

- **Make accuracy**: 38.9% (solidno)
- **Model accuracy**: 0.17% (katastrofalno)  
- **Year accuracy**: 69.0% (dobro)
- **EMR**: 0.00% (potpuni neuspeh)
- **Overall accuracy**: 36.0%
- **Final loss**: 29.88

**Ključni problem**: Model je potpuno neuspešan u prepoznavanju automobila modela, što čini EMR (Exact Match Ratio) nulom jer nijedan uzorak nema sva tri ispravna predviđanja."

In [2]:
# Analiza training history - na osnovu slike sa graficima

# Finalni rezultati nakon 7+ epoha
final_results = {
    'make_accuracy': 0.389,
    'model_accuracy': 0.0017,  # 0.17%
    'year_accuracy': 0.690,
    'make_f1': 0.357,
    'model_f1': 0.0015,
    'year_f1': 0.677,
    'emr': 0.000,
    'overall_accuracy': 0.360,
    'loss': 29.88
}

# Training trends iz grafika
training_trends = {
    'train_loss': 'opadao 17→10',
    'val_loss': 'rastao 25→30', 
    'train_emr': 'rastao 0→0.13',
    'val_emr': 'ostao na 0',
    'make_acc': 'train i val slično ~47%',
    'model_acc': 'train rastao, val ostao na 0'
}

### Problem 1: Katastrofalna model accuracy

**Model accuracy od 0.17%** je praktično identična random pogađanju (0.07%). Ovo znači da model nije uspešno naučio da razlikuje između 1358 različitih modela automobila.

**Uzroci**:
- **Ekstremna nebalansiranost**: Najčešći model (BMW 1_SERIES, 622 slika) vs najređi (2 slike) = 311:1 odnos
- **Previše klasa**: 1358 modela je preambiciozno za dostupne podatke
- **Insufficient regularization**: Model overfittuje na training set ali ne generalizuje

**Posledice**:
- EMR ostaje na 0% jer model komponenta nikad nije tačna
- Training EMR raste ali validation EMR stagnira
- Model uči da uvek predvidi najčešće modele"

In [3]:
# Demonstracija problema sa velikim brojem klasa

# Random baseline za poređenje
num_makes = 81
num_models = 1358 
num_years = 12

random_baseline = {
    'make': 1/num_makes,      # 1.23%
    'model': 1/num_models,    # 0.07% 
    'year': 1/num_years,      # 8.33%
    'emr': (1/num_makes) * (1/num_models) * (1/num_years)  # 0.00001%
}

# Naš model performance
our_model = {
    'make': 0.389,    # 38.9% - dobro
    'model': 0.0017,  # 0.17% - katastrofa
    'year': 0.690,    # 69.0% - odlično
    'emr': 0.000      # 0% - potpun neuspeh
}

# Improvement factor
improvement = {
    'make': our_model['make'] / random_baseline['make'],     # 31.7x
    'model': our_model['model'] / random_baseline['model'],  # 2.4x
    'year': our_model['year'] / random_baseline['year'],     # 8.3x
}

### Problem 2: Overfitting i divergencija loss-a

Iz training history grafika je jasno vidljiva **divergencija između train i validation loss-a**:

- **Training loss**: Kontinuirano opada (17 → 10)
- **Validation loss**: Kontinuirano raste (25 → 30)
- **EMR divergencija**: Training EMR raste do 13%, validation EMR ostaje na 0%

**Uzroci overfittinga**:
- **Nebalansiranost podataka**: Model memorisanje najčešće klase umesto učenja reprezentativnih features
- **Previše parametara**: 26M+ parametara za relativno mali i neuravnotežen dataset
- **Insufficient regularization**: Dropout od 0.5/0.3 nije dovoljno za ovako složen problem

**Indikatori overfittinga**:
- Validation accuracy stagnira dok training accuracy raste
- Loss divergencija je ekstremna (30 vs 10)
- Model se ponaša potpuno drugačije na train vs validation skupu"

In [4]:
# Analiza overfitting patterns iz grafika

# Loss divergencija pattern
loss_pattern = {
    'epoch_0': {'train': 17, 'val': 25},
    'epoch_3': {'train': 13, 'val': 28}, 
    'epoch_7': {'train': 10, 'val': 30},
    'divergence_ratio': 30/10  # 3x razlika na kraju
}

# EMR divergencija pattern  
emr_pattern = {
    'train_emr_max': 0.13,    # 13% na train setu
    'val_emr_max': 0.00,      # 0% na validation setu
    'gap': 0.13               # Potpuna divergencija
}

# Make vs Model accuracy patterns
accuracy_patterns = {
    'make': {'train_val_gap': 'minimalna'},
    'model': {'train_val_gap': 'ekstremna'},  # Train raste, val stagnira
    'year': {'train_val_gap': 'minimalna'}
}

### Problem 3: Multi-output coupling problem

**EMR od 0%** je indikator da model ne može da istovremeno tačno predvidi sva tri izlaza. Čak i kada individualne accuracy nisu loše (make: 39%, year: 69%), kombinacija je katastrofalna.

**Matematička analiza**:
- Teorijski EMR (nezavisni): 0.389 × 0.0017 × 0.690 = 0.046%
- Stvarni EMR: 0.000%
- **Gap**: Model pravi sistemske greške koje sprečavaju kombinovane tačne predikacije

**Multi-output coupling problemi**:
- **Feature interference**: Shared backbone možda ne generiše features pogodne za sve tri zadatka
- **Loss balancing**: Model weight od 2.0 za model loss možda nije dovoljno
- **Training dynamics**: Različiti zadaci se "borse" za backbone parametre tokom treniranja

**Posledica**: Model može biti dobar za individualne zadatke ali lose za kombinovani zadatak koji zahteva EMR metriku."

In [5]:
# EMR analiza - razlaz između teorijskog i stvarnog

# Individualne performanse
individual_acc = {
    'make': 0.389,
    'model': 0.0017, 
    'year': 0.690
}

# Teorijski EMR (ako su predikacije nezavisne)
theoretical_emr = individual_acc['make'] * individual_acc['model'] * individual_acc['year']

# Stvarni EMR
actual_emr = 0.000

# Problem indicators
coupling_problems = {
    'theoretical_emr': theoretical_emr,  # 0.046%
    'actual_emr': actual_emr,            # 0.000%
    'gap': theoretical_emr - actual_emr, # 0.046% missing
    'indicates': 'sistemske greške u kombinovanju predikacija'
}

### Zaključak analize problema

Analiza je identifikovala **sistemske probleme** koji zahtevaju koordinisane izmene:

#### 🚨 Kritični problemi:
1. **Nebalansiranost klasa** - najzastupljeniji model ima 311x više slika od najređeg
2. **Kombinatorička eksplozija** - 1,320,936 teorijskih kombinacija
3. **Severni overfitting** - Validation loss eksplodirala za 640x
4. **EMR = 0.00%** - Model nije uspeo da pogodi sve tri izlaza ni jednom

#### 📊 Ključne metrike (nakon 25 epoha):
- Make accuracy: 39% (1.2% random)
- Model accuracy: 12% (0.07% random)
- Year accuracy: 74% (8.3% random) 
- **EMR: 0.00%** (0.0001% random) 

#### 🔄 Uzročne veze:
Problemi nisu nezavisni - nebalansiranost i veliki broj klasa dovode do lošeg učenja modela i overfittinga, što rezultuje nula EMR metrikom.

#### 💡 Potrebna rešenja:
1. **Konsolidacija modela** - smanjiti broj modela kombinovanjem sličnih
2. **Weighted loss** - boriti se protiv nebalansiranosti
3. **Regularizacija** - sprečiti overfitting
4. **Hijerarhijski pristup** - možda podeli problem na manje delove

**Sledeći korak**: Implementacija rešenja za identifikovane probleme.